# RADx Reporter
This notebook reports studies submitted by the COVID Rapid Acceleration of Diagnostics [RADx Initiative](https://www.nih.gov/research-training/medical-research-initiatives/radx) to the database of Genotypes and Phenotypes [dbGaP](https://www.ncbi.nlm.nih.gov/gap/) and provides a list of data access requests for the following projects:

- [RADx Radical (RADx-rad)](https://www.nih.gov/research-training/medical-research-initiatives/radx/radx-programs#radx-rad)
- [RADx Digital Health Technologies (RADx-DHT)](https://www.nih.gov/news-events/news-releases/nih-awards-contracts-develop-innovative-digital-health-technologies-covid-19)
- [RADx Tech (RADx-TECH)](https://www.nih.gov/research-training/medical-research-initiatives/radx/radx-programs#radx-tech)
- [RADx Underserved Populations (RADx-UP)](https://www.nih.gov/research-training/medical-research-initiatives/radx/radx-programs#radx-up)



For a selected RADx project, it retrieves the studies related to the project and the data access requests. Access requests used to test the RADx Data Hub functionality are excluded by default, but can be optionally included.

The number of access requests per study is a measure of data reuse. In addition, this notebook queries [Europe PMC](https://europepmc.org/) for publications and preprints that cite or mention dbGaP accession numbers.

Created: 2023-03-09

Author : Peter W. Rose (pwrose@ucsd.edu)

In [1]:
#@title Select a RADx project and then select **Run All** from the **Runtime** menu { run: "auto", vertical-output: true, form-width: "50%", display-mode: "form" }
#@markdown ### Select a RADx project
project = 'RADx-rad' #@param ["RADx-rad","RADx-DHT","RADx-TECH","RADx-UP"]
print(f"Project: {project}")

testers = ["Rose, Peter ", "Ciofani, Danielle ", "Krishnamurthy, Ashok ", "Claypool, Kajal ", "Davis-Dusenbery, Brandi "]
#@markdown ### Exclude test requests
exclude_tests = True #@param {type:"boolean"}
print(f"Exclude test requests: {exclude_tests}")

Project: RADx-rad
Exclude test requests: True


In [ ]:
%%capture
#@title Installing software on Google Colab
![ ! -f "installed" ] && pip -q install selenium
![ ! -f "installed" ] && apt-get update
![ ! -f "installed" ] && apt-get install firefox && touch installed

In [ ]:
#@title Importing packages
import os
import shutil
import glob
import time
from tqdm import tqdm
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from google.colab import data_table

In [ ]:
#@title Running query
TMP_DIR = "/tmp"
filepath = os.path.join(TMP_DIR, "studies.csv")

def driversetup(download_dir):
    options = Options()
    #run Selenium in headless mode
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    # https://stackoverflow.com/questions/60170311/how-to-switch-download-directory-using-selenium-firefox-python
    # 0: download to the desktop, 1 download to the default "Downloads" directory, 2 use specified directory
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", download_dir)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/csv")
    
    # https://stackoverflow.com/questions/42204897/how-to-set-up-a-selenium-python-environment-for-firefox
    driver = webdriver.Firefox(options=options)
    driver.implicitly_wait(5)

    return driver

def download_dbgap_studies(query, filepath):
    # clean up any previously downloaded csv files
    files = glob.glob(os.path.join(TMP_DIR, "*.csv"))
    for file in files:
        os.remove(file)
    
    # download csv file
    driver = driversetup(TMP_DIR)
    # for an exact match, put query term in to quotes
    query = '"' + project + '"'
    driver.get(f"https://www.ncbi.nlm.nih.gov/gap/advanced_search/?TERM={query}")
    time.sleep(3)
    #print("Running: ", driver.title)
    button = driver.find_element(By.CLASS_NAME, "svr_container")
    time.sleep(3)
    button.click()
    # wait until download is completed
    for step in tqdm(range(15)):
        time.sleep(1)
    #time.sleep(15)
    driver.close()
                  
    # move downloaded csv file to a standard location
    move_studies_file(filepath)
    
def move_studies_file(filepath):
    """ Move downloaded file to a specified standard location"""
    # the file name of the downloaded csv file is unknown in advance,
    # but there should be only one csv file.
    files = glob.glob(os.path.join(TMP_DIR, "*.csv"))
    if len(files) == 1:
        shutil.move(files[0], filepath)
    else:
        print("query error")
        
filepath = "studies.csv"
download_dbgap_studies(project, filepath)

studies = pd.read_csv(filepath, usecols=["accession", "name", "description", "Study Design", "Study Consent",])
# post filter hits by name field to avoid false matches
studies = studies[studies["name"].str.contains(project, case=False)]

100%|██████████| 15/15 [00:15<00:00,  1.00s/it]


In [ ]:
#@title Table of studies
print(f"Number of studies for {project}:", studies.shape[0])
if studies.shape[0] > 0:
    display(data_table.DataTable(studies, include_index=False, num_rows_per_page=10))

Number of studies for RADx-rad: 48


,accession,name,description,Study Design,Study Consent
0,phs002585.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,This work is directed at characterizing pediat...,Case Set,GRU --- General research use
1,phs002782.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,Infectious disease outbreaks like Coronavirus ...,Case Set,GRU --- General research use
2,phs002679.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,When faced with a pandemic such as SARS-Corona...,Collection,GRU --- General research use
3,phs002525.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,"The University of Miami (UM), with three prima...",Case Set,GRU --- General research use
4,phs002600.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,The data herein combines GC-MS and GC-DMS anal...,Collection,GRU --- General research use
5,phs002603.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,The recent emergence of SARS-CoV2 and resultan...,Prospective Longitudinal Cohort,GRU --- General research use
6,phs002685.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,"Automated, rapid diagnostics with little sampl...",Case Set,GRU --- General research use
7,phs002583.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,We propose to develop a novel testing platform...,Collection,GRU --- General research use
8,phs002604.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,Wastewater based testing (WBT) holds great pro...,Prospective Longitudinal Cohort,GRU --- General research use
9,phs002524.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...,Vulnerable populations do not just need testin...,Collection,GRU --- General research use


In [ ]:
#@title Summary of data access requests
def get_download_url(accession):
    return "https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/GetAuthorizedRequestDownload.cgi?study_id=" + accession

def get_authorized_requests(studies):
    authorized_requests = pd.DataFrame()

    for _, row in tqdm(studies.iterrows(), total=studies.shape[0]):
        try:
            df = pd.read_csv(get_download_url(row["accession"]), 
                             usecols=["Requestor", "Affiliation", "Project", "Date of approval", "Request status", 
                                      "Public Research Use Statement", "Technical Research Use Statement"],
                            sep="\t")
            df["accession"] = row["accession"]
            df["name"] = row["name"]
            authorized_requests = pd.concat([authorized_requests, df], ignore_index=True)
        except:
            print(f"Skipping: {row['accession']} - no data access through dbGaP.")
                                        
    return authorized_requests

requests = get_authorized_requests(studies)

# exclude test requests
n_excluded = 0
if exclude_tests:
    n_excluded = requests.shape[0]
    requests = requests[~requests["Requestor"].isin(testers)]
    n_excluded = n_excluded - requests.shape[0]


# group requests to create a summary view
if requests.shape[0] > 0:
    summary = requests.groupby(["Requestor", "Affiliation", "Project", "Date of approval", "Request status",
                                "Public Research Use Statement", "Technical Research Use Statement"], 
                                as_index=False)["accession"].agg(', '.join)

    summary["Number of requests"] = summary["accession"].str.count(",") + 1

# show results
print()
print()
if exclude_tests:
        print(f"{n_excluded} test requests have been excluded!")
        
print("Number of data access requests :", requests.shape[0])

if requests.shape[0] > 0:
    print("Number of unique requestors    :", len(requests["Requestor"].unique()))
    print("Number of unique studies       :", len(requests["accession"].unique()))
    display(data_table.DataTable(summary, include_index=False, num_rows_per_page=10))

100%|██████████| 48/48 [01:00<00:00,  1.25s/it]



49 test requests have been excluded!
Number of data access requests : 3
Number of unique requestors    : 2
Number of unique studies       : 3


,Requestor,Affiliation,Project,Date of approval,Request status,Public Research Use Statement,Technical Research Use Statement,accession,Number of requests
0,"Anwar, Mohd Mozharul",NIH,Exploration of Wearable Device Data for COVID-19,"Mar27, 2023",approved,Wearable devices collect various physiological...,The objective of the proposed research is to e...,phs002523.v1.p1,1
1,"Miguez, Maria-Jose",FLORIDA INTERNATIONAL UNIVERSITY,System analysis for COVID humoral response,"Feb10, 2023",approved,Multisystem inflammatory syndrome in children ...,Multisystem inflammatory syndrome in children ...,"phs002781.v1.p1, phs002945.v1.p1",2


In [ ]:
#@title Detailed table of data access requests
if exclude_tests:
    print(f"{n_excluded} test requests have been excluded!")

print("Number of data access requests :", requests.shape[0])

if requests.shape[0] > 0:
    print("Number of unique requestors    :", len(requests["Requestor"].unique()))
    print("Number of unique studies       :", len(requests["accession"].unique()))
    display(data_table.DataTable(requests, include_index=False, num_rows_per_page=10))

49 test requests have been excluded!
Number of data access requests : 3
Number of unique requestors    : 2
Number of unique studies       : 3


,Requestor,Affiliation,Project,Date of approval,Request status,Public Research Use Statement,Technical Research Use Statement,accession,name
19,"Anwar, Mohd Mozharul",NIH,Exploration of Wearable Device Data for COVID-19,"Mar27, 2023",approved,Wearable devices collect various physiological...,The objective of the proposed research is to e...,phs002523.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...
46,"Miguez, Maria-Jose",FLORIDA INTERNATIONAL UNIVERSITY,System analysis for COVID humoral response,"Feb10, 2023",approved,Multisystem inflammatory syndrome in children ...,Multisystem inflammatory syndrome in children ...,phs002781.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...
50,"Miguez, Maria-Jose",FLORIDA INTERNATIONAL UNIVERSITY,System analysis for COVID humoral response,"Feb10, 2023",approved,Multisystem inflammatory syndrome in children ...,Multisystem inflammatory syndrome in children ...,phs002945.v1.p1,Rapid Acceleration of Diagnostics - Radical (R...


In [ ]:
#@title Publications that cite or mention dbGaP accession numbers
studies["dbgap"] = studies["accession"].apply(lambda s: s.split(".")[0])
# get list of publications from Europe PMC
dbgap_pub = pd.read_csv("ftp://ftp.ebi.ac.uk/pub/databases/pmc/TextMinedTerms/dbgap.csv")
pubs = studies.merge(dbgap_pub, on="dbgap")
print("Number of publications:", pubs.shape[0])
display(data_table.DataTable(pubs, include_index=False, num_rows_per_page=10))

Number of publications: 0


,accession,name,description,Study Design,Study Consent,dbgap,PMCID,EXTID,SOURCE


In [7]:
#@title Publications indexed by Europe PMC
# import urllib library
from urllib.request import urlopen

# import json
import json
# store the URL in url as
# parameter for urlopen
url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/search?query={project}%20AND%20(FIRST_PDATE:%5b2020-01-01%20)%20AND%20AND%20%20sort_date:y&resultType=idlist&pageSize=1000&format=json&cursorMark=*"

# store the response of URL
response = urlopen(url)

# storing the JSON response
# from url in data
data_json = json.loads(response.read())

# print the json response
print(data_json["resultList"]["result"])


[{'id': '36965380', 'source': 'MED', 'pmid': '36965380', 'pmcid': 'PMC10027305', 'fullTextIdList': {'fullTextId': ['PMC10027305']}}, {'id': '36933724', 'source': 'MED', 'pmid': '36933724', 'pmcid': 'PMC10017378', 'fullTextIdList': {'fullTextId': ['PMC10017378']}}, {'id': '36832064', 'source': 'MED', 'pmid': '36832064', 'pmcid': 'PMC9954708', 'fullTextIdList': {'fullTextId': ['PMC9954708']}}, {'id': '36623667', 'source': 'MED', 'pmid': '36623667', 'pmcid': 'PMC9817413', 'fullTextIdList': {'fullTextId': ['PMC9817413']}}, {'id': '36560869', 'source': 'MED', 'pmid': '36560869', 'pmcid': 'PMC9790855', 'fullTextIdList': {'fullTextId': ['PMC9790855']}}, {'id': '36590003', 'source': 'MED', 'pmid': '36590003', 'pmcid': 'PMC9798920', 'fullTextIdList': {'fullTextId': ['PMC9798920']}}, {'id': '36482179', 'source': 'MED', 'pmid': '36482179', 'pmcid': 'PMC9731983', 'fullTextIdList': {'fullTextId': ['PMC9731983']}}, {'id': '36493788', 'source': 'MED', 'pmid': '36493788', 'pmcid': 'PMC9725778', 'fullT